## Imports

In [1]:
import os

## Config

In [2]:
feature_list_id = 'simple_summaries'

In [3]:
data_folder = os.path.abspath(os.path.join(os.curdir, os.pardir, 'data')) + os.path.sep
aux_data_folder = os.path.join(data_folder, 'aux') + os.path.sep
preproc_data_folder = os.path.join(data_folder, 'preproc') + os.path.sep
features_data_folder = os.path.join(data_folder, 'features') + os.path.sep

## Read Data

In [4]:
df_questions_train = pd.read_csv(data_folder + 'train.csv').fillna('')
df_questions_test = pd.read_csv(data_folder + 'test.csv').fillna('')

In [5]:
question_tokens_train = load_json(preproc_data_folder + 'question_tokens_train.json')
question_tokens_test = load_json(preproc_data_folder + 'question_tokens_test.json')

## Build Features

In [6]:
def word_share_ratio(q1_tokens, q2_tokens):
    return 2 * len(set(q1_tokens) & set(q2_tokens)) / (len(set(q1_tokens)) + len(set(q2_tokens)))

In [7]:
def word_difference_ratio(q1_tokens, q2_tokens):
    return len(set(q1_tokens) ^ set(q2_tokens)) / (len(set(q1_tokens)) + len(set(q2_tokens)))

In [8]:
def build_features(df_questions_original, questions_tokenized):
    num_pairs = len(questions_tokenized)
    num_features = 8
    
    X = np.zeros((num_pairs, num_features), dtype=float)
    
    for index, pair in progressbar(enumerate(questions_tokenized), size=num_pairs):
        q1_original = df_questions_original.ix[pair['id']].question1
        q2_original = df_questions_original.ix[pair['id']].question2

        # Length of questions (in characters)
        X[index, 0] = len(q1_original)
        X[index, 1] = len(q2_original)

        # Length of questions (in tokens)
        X[index, 2] = len(pair['question1'])
        X[index, 3] = len(pair['question2'])

        # Difference of question length (in characters)
        X[index, 4] = abs(len(q1_original) - len(q2_original))
        
        # Difference of question length (in tokens)
        X[index, 5] = abs(len(pair['question1']) - len(pair['question2']))
        
        # Word share ratio for question tokens.
        X[index, 6] = word_share_ratio(pair['question1'], pair['question2'])

        # Word difference ratio for question tokens.
        X[index, 7] = word_difference_ratio(pair['question1'], pair['question2'])
        
    return X

In [9]:
X_train = build_features(df_questions_train, question_tokens_train)

Widget Javascript not detected.  It may not be installed or enabled properly.


In [10]:
save(X_train, features_data_folder + f'X_train_{feature_list_id}.pickle')

In [11]:
X_test = build_features(df_questions_test, question_tokens_test)

Widget Javascript not detected.  It may not be installed or enabled properly.


In [12]:
save(X_test, features_data_folder + f'X_test_{feature_list_id}.pickle')